# Sentence Window Retrieval - AI Training

In [ ]:
import os
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
)
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.indices.postprocessor import (
    MetadataReplacementPostProcessor,
    SentenceTransformerRerank,
)
from llama_index.core import load_index_from_storage
from llama_index.llms.openai import OpenAI


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # Create node parser
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )

    # Set global settings (this replaces ServiceContext)
    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.node_parser = node_parser

    # Create or load the index
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir)
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # Postprocessor to insert full sentence windows
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

    # Reranker to reorder top results using a smarter model
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    # Build query engine with both enhancements
    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k,
        node_postprocessors=[postproc, rerank],
    )

    return sentence_window_engine


In [127]:
from llama_index.llms.openai import OpenAI 
from llama_index.core import Settings

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./XC1-Character-Guide.pdf"]
).load_data()


In [128]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index",
)

In [ ]:
from llama_index.core.response.notebook_utils import display_response

query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

with open('./generated_questions.txt') as file:
    for line in file:
        question = line.strip()
        print(question)
        display_response(query_engine.query(question))




My favorite team is Fiora (Seven), Melia, and Riki, where I control Melia. What are the pros and cons of this team? Consider the AI of the non-player controlled units, chain attacks, buffs, debuffs, and overall synergy.


**`Final Response:`** The team of Fiora (Seven), Melia, and Riki has several pros and cons. One advantage is that Seven can function well as a tank, drawing aggro and providing survivability support for Melia, who has low survivability. Riki, as mentioned, is rated as one of the best teammates for Melia due to their synergy. Additionally, Melia's flexibility with passive buffs and in Chain Attacks can benefit the team overall.

However, a potential drawback is that Melia struggles with filling the Party Gauge efficiently, which can hinder the team's Chain Attack potential. Additionally, while Seven and Melia have good synergy, Riki's AI-controlled healing abilities may not always align perfectly with the team's needs, especially in terms of generating aggro or maximizing Chain Attack damage.

Overall, the team of Fiora (Seven), Melia, and Riki can work well together due to their individual strengths and synergies, but may face challenges in optimizing Chain Attacks and Party Gauge filling efficiency.

What is the best possible party in Xenoblade Chronicles. Consider all possible teams?


**`Final Response:`** The best possible party in Xenoblade Chronicles would likely include Seven and Shulk due to their high damage output, powerful Arts, and unique abilities that make them versatile and effective in various situations. Additionally, Seven's ability to damage Mechon without requiring Enchant and Shulk's positioning-dependent Arts add strategic depth to the party composition.

Why is Sharla considered the worst party member? Can you argue for her placement as the best?


**`Final Response:`** Sharla is considered the worst party member due to her low DPS, lack of synergy with certain teammates like Reyn and Dunban, and her limitations in building the Party Gauge effectively. She hinders damage output and Chain Attacks with certain party compositions, making her less desirable in many situations. However, some may argue for her placement as the best due to her sustainability and ability to provide healing and support, which can be crucial for long-term battles and keeping the party alive without additional assistance.

Why is Reyn considered the best party member? Can you argue for his placement as the best?


**`Final Response:`** Reyn is considered the best party member due to his fantastic party support capabilities, including debuffs like Paralysis, Agility Down, and Strength Down, which significantly reduce enemy damage output. He also excels in AoE damage and aggro draw, possesses incredible Burst damage potential, and can reach high dodge rates despite his low Agility. While he has weaknesses like low sustained damage, his overall potential is often underestimated, making a strong case for his placement as the best party member.